## Library Import

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()





In [ ]:
import geemap
import numpy as np
import matplotlib as plt
import pandas as pd
import epistemx.module_1

## Helper Functions

In [ ]:
#############################  Area of Interest  ###########################
def get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan"):
    """
    Get Area of Interest geometry from GAUL administrative boundaries.
    
    Parameters:
    -----------
    country : str
        Country name (default: "Indonesia")
    province : str
        Province/state name (default: "Sumatera Selatan")
        
    Returns:
    --------
    ee.Geometry : Area of interest geometry
    """
    admin = ee.FeatureCollection("FAO/GAUL/2015/level1")
    aoi_fc = admin.filter(ee.Filter.eq('ADM0_NAME', country)).filter(
        ee.Filter.eq('ADM1_NAME', province)
    )
    return aoi_fc.geometry()

In [ ]:
# --- Area of Interest (AOI) ---
# Set the country and province for the AOI using GAUL admin boundaries
COUNTRY = "Indonesia"
PROVINCE = "Sumatera Selatan"
aoi = get_aoi_from_gaul(country=COUNTRY, province=PROVINCE)
#Alternatively, used geemap_shp_to_ee to directly used shapefile in your local machine
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2018-12-31'
#get the image collection and corresponding statistics
landsat_data, stats = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=False)
#visualization parameter
l8_sr_visparam = {
    'min': 0,
    'max': 0.4,
    'gamma': [0.95, 1.1, 1],
    'bands':['NIR', 'RED', 'GREEN']
}
#create mosaic between image collection, sort them based on the cloud cover property, and clip based on AOI
mosaic_landsat = landsat_data.sort('CLOUD_COVER_LAND').mosaic().clip(aoi)
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map 
# set center of the map in the area of interest
Map.centerObject(aoi, 7)
# adding the data
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')